# 第四章 决策树

## 4.1 试证明对于不含冲突数据(即特征向量完全相同但标记不同)的训练集，必存在与训练集一致(即训练误差为 0) 的决策树。

若数据集本身无误，决策树理论上也无误。

我们对某次判断作终止条件的分析：

（1）当前结点包含样本全属于同一类别；

（2）当前属性集为空，或所有样本在所有属性上取值相同；

（3）当前结点包含样本集为空。

显然，若判断终止，（2）时，若当前属性集为空则取标记多的为分类结果，无误差，若样本在所有属性上取值相同，因数据不冲突，故必为同类，等价于（1），（1）时，分类完毕，无误差，（3）回溯上一次判断的情况，必然属于（1）（2）（3）中一种。

故不含冲突数据的训练集必存在无训练误差决策树。

## 4.2 试析使用"最小训练误差"作为决策树划分选择准则的缺陷。

模型泛化能力较差。类似于多元回归模型，有些变量应该被剔除，决策树也存在过拟合问题，即不仅学习了数据的泛化特征，也学习了训练集本身特点。

## 4.3 试编程实现基于信息熵进行划分选择的决策树算法，并为表 4.3 中数据生成一棵决策树。

参考：

[机器学习(周志华) 参考答案 第四章 决策树 python重写版与画树算法](https://blog.csdn.net/icefire_tyh/article/details/54575527)

[决策树(Decision Tree) | 算法实现](https://blog.csdn.net/sudden2012/article/details/82810899)

[Python实现树结构](https://blog.csdn.net/m0_37324740/article/details/79435814)

[Python数据结构应用6——树](https://www.cnblogs.com/bjwu/p/9016566.html)

[如何实现Python多叉树](https://zhidao.baidu.com/question/1817955110065615148.html)

[python-去除二维数组/二维列表中的重复行](https://blog.csdn.net/u012991043/article/details/81067207)

PS:这一章就没啥公式，全在看数据结构与算法，总感觉我写的树结构怪怪的。写到一半发现df切片挺麻烦的，还是应该写一个melon类，但是懒得改了，以后再整理。

依据信息熵增益选择最优属性那部分需要完善，当前只是根据单一准则如Gain进行最优选择，没有写启发式。

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 提取数据
read_path = 'D:\JWE\Files\Courses\数据挖掘\melon_data.csv'
melon_data = pd.read_csv(read_path)
# melon_data.columns
# data0 = melon_data.drop(columns=['编号'])
data0 = melon_data[:]
# T_midu = round(float(data0.loc[:,['密度']].median().values),4)
# T_hantang = round(float(data0.loc[:,['含糖率']].median().values),4)
T_midu = 0.381 # 书上取划分点0.381，0.126
T_hantang = 0.126 
data0.loc[data0['好瓜'] == '是',['好瓜']] = '好瓜'
data0.loc[data0['好瓜'] == '否',['好瓜']] = '坏瓜'
data0.loc[melon_data['密度'] > T_midu,['密度']],data0.loc[melon_data['密度'] <= T_midu,['密度']] = f'大于{T_midu}',f'小于{T_midu}'
data0.loc[melon_data['含糖率'] > T_hantang,['含糖率']],data0.loc[melon_data['含糖率'] <= T_hantang,['含糖率']] = f'大于{T_hantang}',f'小于{T_hantang}'
# data0
character_dict = {character:attr for character,attr in zip(data0.columns.drop(['编号','好瓜']),[set(data0.loc[:,[each]].values.flatten()) for each in data0.columns.drop(['编号','好瓜'])])}
# character_dict
data1 = data0.drop(columns=['密度','含糖率'])

In [3]:
# 未实装
class melon(object):
    def __init__(self,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率):
        self.编号 = 编号
        self.色泽 = 色泽
        self.根蒂 = 根蒂
        self.敲声 = 敲声
        self.纹理 = 纹理
        self.脐部 = 脐部
        self.触感 = 触感
        self.密度 = 密度
        self.含糖率 = 含糖率
    def __str__(self):
        return '<' + str(self.编号) + ',' + str(self.色泽) + ',' + str(self.根蒂) + ',' + str(self.敲声)  + ',' + str(self.纹理) + ',' + str(self.脐部) + ',' + str(self.触感) + ',' + str(self.密度) + ','+ str(self.含糖率)+ '>'
    
def melon_case(data):
    return [melon(编号=each[0],色泽=each[1],根蒂=each[2],敲声=each[3],纹理=each[4],脐部=each[5],触感=each[6],密度=each[7],含糖率=each[8]) for each in data.values]

In [4]:
# 划分指标
characters = data0.columns
classi = '好瓜' # classi = '好瓜'
precision = 3
def Ent(data,character=classi,value=None,classi=classi,precision=precision):
    Ent = 0
    if character == classi:
        D = data.loc[:,[classi]]
        for each in set(data.loc[:,[classi]].values.flatten()):
            p = len(data.loc[data[classi]==each,[classi]].values)/len(D.values)
            Ent += -p*np.log2(p)
    else:
        D = data.loc[data[character]==value,[classi]]
        for each in set(data.loc[data[character]==value,[classi]].values.flatten()):
            p = len(data.loc[(data[character]==value)&(data[classi]==each),[classi]].values)/len(D.values)
            Ent += -p*np.log2(p)
    return  round(Ent,precision)
    
def Gain(data,character=classi,classi=classi,precision=precision):
    Gain = Ent(data)
    for value in set(data.loc[:,[character]].values.flatten()):
        D = len(data)
        D_V = len(data.loc[(data[character]==value),[classi]])
        Gain += -(D_V/D)*Ent(data,character,value)
    return round(Gain,precision)

def IV(data,character,classi=classi):
    IV = 0
    for value in set(data.loc[:,[character]].values.flatten()):    
        D = len(data)
        D_V = len(data.loc[(data[character]==value),[classi]])
        IV += -(D_V/D)*np.log2(D_V/D)
    return IV

def Gain_ratio(data,character,classi=classi,precision=precision):
    Gain_ = Gain(data,character,classi=classi)
    IV_ = IV(data,character,classi=classi)
    eps = 10**(-4)
    if IV_ == 0:
        Gain_ratio = Gain_/eps
    else:
        Gain_ratio = Gain_/IV_
    return round(Gain_ratio,precision)

# 基尼系数
def Gini(data,classi=classi,precision=6):
    Gini = 1
    D = data.loc[:,[classi]]
    for each in set(data.loc[:,[classi]].values.flatten()):
        p = len(data.loc[data[classi]==each,[classi]].values)/len(D.values)
        Gini += -p**2
    return round(Gini,6)

def Gini_index(data,character,classi=classi,precision=precision):
    Gini_a = 0
    for value in set(data.loc[:,[character]].values.flatten()):
        D = data
        D_V = data.loc[data[character]==value,:]
        Gini_a += (len(D_V)/len(D))*Gini(D_V)    
    return round(Gini_a,precision)

In [5]:
# 决策树生成
class Node(object):
    def __init__(self,root=None,maxcnum=10**4):
        self.root = root
        self.children = []
        self.max_children_num = maxcnum
    def __str__(self):
        return '[' + str(self.root) + ',' + str([each.root for each in self.children]) + ']'

class Tree(object):
    def __init__(self):
        self.root = None
    def add(self,node):
        if self.root is None:
            self.root = node
            return
        queue = [self.root]
        while queue:
            cur_node = queue.pop(0)
            if len(cur_node.children) < cur_node.max_children_num:
                cur_node.children.append(node)
                return
            else:
                for each in cur_node.children:
                    queue.append(each)
    def breadth_travel(self):
        if self.root is None:
            return
        queue = [self.root]
        while queue:
            cur_node = queue.pop(0)
            print(cur_node.root)
            if cur_node.children:
                for each in cur_node.children:
                    queue.append(each)
    def preorder(self,node):
        if node is None:
            return
        print(node.root)
        for each in node.children:
            self.preorder(each)      

def decision_class(data):
    value_list = list(set(data.loc[:,['好瓜']].values.flatten()))
    if len(value_list) >= 2:
        class_dict = {value_list[0]:len(data.loc[data['好瓜']==value_list[0],['好瓜']]),value_list[1]:len(data.loc[data['好瓜']==value_list[1],['好瓜']])}
        dclass = max(class_dict,key=class_dict.get)
        return dclass
    else:
        return value_list[0]            

# 深度优先生成决策树,未剪枝
# divide_fun为划分指标，Gain,Gain_ratio,Gini_index
def TreeGenerate(data,character_set,tree=Tree(),divide_fun=Gain):
    # 生成node 
    dnode = Node(root={'temp':list(data.loc[:,['编号']].values.flatten())},maxcnum=1)
    # D中样本属于同一类别C
    if len(set(data.loc[:,['好瓜']].values.flatten())) == 1: 
        # 将node标记为C类叶结点
        dnode.root[list(set(data.loc[:,['好瓜']].values.flatten()))[0]] = dnode.root.pop('temp')
        tree.add(dnode)
        return 
    # character_set为空 or data中样本在attr上取值相同
    if len(character_set) == 0 or len(set([tuple(each) for each in data.loc[:,list(character_set)].values])) <= 1: 
        # 将node标记为叶结点，其类别标记为D中样本数最多的类
        dnode.root[decision_class(data)] = dnode.root.pop('temp')
        tree.add(dnode)
        return 
    # 从A中选择最优划分属性attr_best
    character_dict = {character:attr for character,attr in zip(character_set,[set(data.loc[:,[each]].values.flatten()) for each in character_set])}
    gain_dict = {character:divide_fun(data,character) for character in character_set} 
    attr_best = max(gain_dict,key=gain_dict.get)
    if divide_fun == Gini_index:
        attr_best = min(gain_dict,key=gain_dict.get)
    anode = Node(root=f'{attr_best}',maxcnum=len(character_dict[attr_best]))
    tree.add(anode)
    # 遍历最优划分属性attr_best的取值
    for attr in character_dict[attr_best]:
        # 为node生成一个分支
        tnode = Node(root={f'{attr_best}={attr}':list(data.loc[data[attr_best]==attr,['编号']].values.flatten())},maxcnum=1)
        # 令D_v表示D在attr上取值为attr_v的样本子集
        attr_index = [i-1 for i in data.loc[data[attr_best]==attr,['编号']].values.flatten()]
        data_v = data.loc[attr_index,:]
        if len(data_v) == 0: # D_v为空
            # 将分支结点标记为叶结点，其类别标记为D中样本最多的类
            tnode.root[decision_class(data)] = tnode.root.pop(f'{attr_best}={attr}')
            tree.add(tnode)
            return 
        else:
            # 以TreeGenerate(D_v,characters\{attr_best})为分支结点
            tree.add(tnode)
            character_set = list(set(character_set) - {attr_best})
            TreeGenerate(data=data_v,character_set=character_set,tree=tree,divide_fun=divide_fun)

character_set0 = list(data0.columns.drop(['编号','好瓜']))
def main(data=data0,character_set=character_set0,divide_fun=Gain):
    tree = Tree()
    TreeGenerate(data,character_set,tree=tree,divide_fun=divide_fun)
    tree.breadth_travel() 
#     tree.preorder(tree.root)
    return tree

In [6]:
list({'根蒂','密度','脐部'} - {'根蒂'})
list(set(character_set0) - {'根蒂'})
list(set(character_set0) - set('根蒂'))

['密度', '脐部']

['脐部', '含糖率', '触感', '敲声', '密度', '色泽', '纹理']

['根蒂', '脐部', '含糖率', '触感', '敲声', '密度', '色泽', '纹理']

In [7]:
# 标记为类别的就是叶结点
main(divide_fun=Gain)
main(divide_fun=Gain_ratio)

纹理
{'纹理=稍糊': [7, 9, 13, 14, 17]}
触感
{'触感=硬滑': [9, 13, 14, 17]}
{'坏瓜': [9, 13, 14, 17]}
{'触感=软粘': [7]}
{'好瓜': [7]}
{'纹理=模糊': [11, 12, 16]}
{'坏瓜': [11, 12, 16]}
{'纹理=清晰': [1, 2, 3, 4, 5, 6, 8, 10, 15]}
密度
{'密度=大于0.381': [1, 2, 3, 4, 5, 6, 8]}
{'好瓜': [1, 2, 3, 4, 5, 6, 8]}
{'密度=小于0.381': [10, 15]}
{'坏瓜': [10, 15]}


含糖率
{'含糖率=小于0.126': [9, 11, 12, 16, 17]}
{'坏瓜': [9, 11, 12, 16, 17]}
{'含糖率=大于0.126': [1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 14, 15]}
密度
{'密度=大于0.381': [1, 2, 3, 4, 5, 6, 7, 8, 13, 14]}
纹理
{'纹理=稍糊': [7, 13, 14]}
脐部
{'脐部=凹陷': [13, 14]}
{'坏瓜': [13, 14]}
{'脐部=稍凹': [7]}
{'好瓜': [7]}
{'纹理=清晰': [1, 2, 3, 4, 5, 6, 8]}
{'好瓜': [1, 2, 3, 4, 5, 6, 8]}
{'密度=小于0.381': [10, 15]}
{'坏瓜': [10, 15]}


## 4.4 试编程实现基于基尼指数进行划分选择的决策树算法，为表 4.2 中数据生成预剪枝、后剪枝决策树，并与未剪枝决策树进行比较。

沿用上面代码

In [8]:
# 基尼系数
def Gini(data,classi=classi,precision=6):
    Gini = 1
    D = data.loc[:,[classi]]
    for each in set(data.loc[:,[classi]].values.flatten()):
        p = len(data.loc[data[classi]==each,[classi]].values)/len(D.values)
        Gini += -p**2
    return round(Gini,6)

def Gini_index(data,character,classi=classi,precision=precision):
    Gini_a = 0
    for value in set(data.loc[:,[character]].values.flatten()):
        D = data
        D_V = data.loc[data[character]==value,:]
        Gini_a += (len(D_V)/len(D))*Gini(D_V)    
    return round(Gini_a,precision)

In [9]:
main(divide_fun=Gini_index)

纹理
{'纹理=稍糊': [7, 9, 13, 14, 17]}
触感
{'触感=硬滑': [9, 13, 14, 17]}
{'坏瓜': [9, 13, 14, 17]}
{'触感=软粘': [7]}
{'好瓜': [7]}
{'纹理=模糊': [11, 12, 16]}
{'坏瓜': [11, 12, 16]}
{'纹理=清晰': [1, 2, 3, 4, 5, 6, 8, 10, 15]}
密度
{'密度=大于0.381': [1, 2, 3, 4, 5, 6, 8]}
{'好瓜': [1, 2, 3, 4, 5, 6, 8]}
{'密度=小于0.381': [10, 15]}
{'坏瓜': [10, 15]}


In [10]:
# 深度优先生成决策树,预剪枝
# divide_fun为划分指标，Gain,Gain_ratio,Gini_index
def TreeGenerateWithPreCut(data_train,data_test,character_set,tree=Tree(),divide_fun=Gain):
    # 生成node 
    dnode = Node(root={'temp':list(data_train.loc[:,['编号']].values.flatten())})
    # D中样本属于同一类别C
    if len(set(data_train.loc[:,['好瓜']].values.flatten())) <= 1: 
        # 将node标记为C类叶结点
        dnode.root[list(set(data_train.loc[:,['好瓜']].values.flatten()))[0]] = dnode.root.pop('temp')
        tree.add(dnode)
        return 
    # 从A中选择最优划分属性attr_best
    character_dict = {character:attr for character,attr in zip(character_set,[set(data_train.loc[:,[each]].values.flatten()) for each in character_set])}
    gain_dict = {character:divide_fun(data_train,character) for character in character_set} 
    attr_best = max(gain_dict,key=gain_dict.get)
    if divide_fun == Gini_index:
        attr_best = min(gain_dict,key=gain_dict.get)
    # 计算验证集上划分前后的精度accuracy
    dclass = decision_class(data_train)
    pre_class_dict = {f'{attr}':decision_class(data_train.loc[data_train[attr_best]==attr,:]) for attr in character_dict[attr_best]}
    accuracy_pre = len(data_test.loc[data_test.loc[:,'好瓜']==dclass,:])/len(data_test)
    correct_aft = [len(data_test.loc[(data_test[attr_best]==attr)&(data_test['好瓜']==dclass),:]) for attr,dclass in zip(pre_class_dict.keys(),pre_class_dict.values())]
    accuracy_aft = sum(correct_aft)/len(data_test)
    # character_set为空 or data中样本在attr上取值相同 or 验证集划分前精度不小于划分后精度
    if len(character_set) == 0 or len(set([tuple(each) for each in data_train.loc[:,list(character_set)].values])) <= 1 or accuracy_pre >= accuracy_aft: 
        # 将node标记为叶结点，其类别标记为D中样本数最多的类
        dnode.root[decision_class(data_train)] = dnode.root.pop('temp')
        tree.add(dnode)
        return 
    # 选择最优划分属性attr_best为分支结点
    anode = Node(root=f'{attr_best}')
    tree.add(anode)
    # 遍历最优划分属性attr_best的取值
    for attr in character_dict[attr_best]:
        # 为node生成一个分支
        tnode = Node(root={f'{attr_best}={attr}':list(data_train.loc[data_train[attr_best]==attr,['编号']].values.flatten())})
        # 令D_v表示D在attr上取值为attr_v的样本子集
        attr_index = [i-1 for i in data_train.loc[data_train[attr_best]==attr,['编号']].values.flatten()]
        data_v = data_train.loc[attr_index,:]
        if len(data_v) == 0: # D_v为空
            # 将分支结点标记为叶结点，其类别标记为D中样本最多的类
            tnode.root[decision_class(data_train)] = tnode.root.pop(f'{attr_best}={attr}')
            tree.add(tnode)
            return 
        else:
            # 以TreeGenerate(D_v,characters\{attr})为分支结点
            tree.add(tnode)
            character_set1 = list(set(character_set) - {attr_best})
            TreeGenerateWithPreCut(data_train=data_v,data_test=data_test,character_set=character_set1,tree=tree,divide_fun=divide_fun)

In [11]:
# data1
train_index = [1,2,3,6,7,10,14,15,16,17]
test_index = [4,5,8,9,11,12,13]
data_train = data1.loc[[i-1 for i in train_index],:]
data_test = data1.loc[[i-1 for i in test_index],:]
character_set1 = list(data1.columns.drop(['编号','好瓜']))
def PreCutTree(data_train=data_train,data_test=data_test,character_set=character_set1,divide_fun=Gini_index):
    character_set1 = list(data1.columns.drop(['编号','好瓜']))
    tree = Tree()
    TreeGenerateWithPreCut(data_train=data_train,data_test=data_test,character_set=character_set1,tree=tree,divide_fun=divide_fun)
    tree.breadth_travel() 
    return tree

In [12]:
# 预减枝与未剪枝
divide = Gain
PreCutTree(data_train=data_train,data_test=data_test,divide_fun=divide)
print()
main(data=data1,character_set=character_set1,divide_fun=Gain_ratio)

色泽
{'色泽=乌黑': [2, 3, 7, 15]}
{'好瓜': [2, 3, 7, 15]}
{'色泽=浅白': [14, 16]}
{'坏瓜': [14, 16]}
{'色泽=青绿': [1, 6, 10, 17]}
敲声
{'敲声=清脆': [10]}
{'坏瓜': [10]}
{'敲声=沉闷': [17]}
{'坏瓜': [17]}
{'敲声=浊响': [1, 6]}
{'好瓜': [1, 6]}



纹理
{'纹理=稍糊': [7, 9, 13, 14, 17]}
触感
{'触感=硬滑': [9, 13, 14, 17]}
{'坏瓜': [9, 13, 14, 17]}
{'触感=软粘': [7]}
{'好瓜': [7]}
{'纹理=模糊': [11, 12, 16]}
{'坏瓜': [11, 12, 16]}
{'纹理=清晰': [1, 2, 3, 4, 5, 6, 8, 10, 15]}
触感
{'触感=硬滑': [1, 2, 3, 4, 5, 8]}
{'好瓜': [1, 2, 3, 4, 5, 8]}
{'触感=软粘': [6, 10, 15]}
根蒂
{'根蒂=稍蜷': [6, 15]}
色泽
{'色泽=乌黑': [15]}
{'坏瓜': [15]}
{'色泽=青绿': [6]}
{'好瓜': [6]}
{'根蒂=硬挺': [10]}
{'坏瓜': [10]}


## 4.5 试编程实现基于对率回归进行划分选择的决策树算法，并为表 4.3数据生成一棵决策树.

沿用上面代码，日后再补。

## 4.6 试选择 UCI 数据集，对上述 种算法所产生的未剪枝、预剪枝、后剪枝决策树进行实验比较，并进行适当的统计显著性检验。

日后再补

## 4.7 图4.2 是一个递归算法，若面临巨量数据，则决策树的层数会很深，使用递归方法易导致"栈"溢出。试使用"队列"数据结构，以参数MaxDepth控制树的最大深度，写出与图 4.2 等价、但不使用递归的决策树生成算法。

沿用上面代码，用队列的话应该只能写广度优先吧，但题目说与4.2等价那应该是深度优先，应该用栈。写完懒得改了，栈和队列的区别就是先进后出和先进先出。

In [13]:
# 广度优先生成决策树，队列控制深度
def TreeGenerateByQueue(data,tree=Tree(),divide_fun=Gain,MaxDepth=10**4):
    character_set = list(data.columns.drop(['编号','好瓜']))
    # 队列
    queue = [data]
    # 深度
    depth = 0
    while queue:
#         print(len(queue)) # 调试1
        data = queue.pop(0)
#         print(character_set) # 调试2
        # 生成node 
        dnode = Node(root={'temp':list(data.loc[:,['编号']].values.flatten())}) # maxcnum=1
        # D中样本属于同一类别C
        if len(set(data.loc[:,['好瓜']].values.flatten())) == 1: 
            # 将node标记为C类叶结点
            dnode.root[list(set(data.loc[:,['好瓜']].values.flatten()))[0]] = dnode.root.pop('temp')
            tree.add(dnode)
            continue
#             return 
        # character_set为空 or data中样本在attr上取值相同
        if len(character_set) == 0 or len(set([tuple(each) for each in data.loc[:,list(character_set)].values])) <= 1: 
            # 将node标记为叶结点，其类别标记为D中样本数最多的类
            dnode.root[decision_class(data)] = dnode.root.pop('temp')
            tree.add(dnode)
            continue
#             return 
        if depth == MaxDepth:
            continue
        # 深度加1
        depth += 1
#         print(depth) # 调试3
        # 从A中选择最优划分属性attr_best
        character_dict = {character:attr for character,attr in zip(character_set,[set(data.loc[:,[each]].values.flatten()) for each in character_set])}
        gain_dict = {character:divide_fun(data,character) for character in character_set} 
        attr_best = max(gain_dict,key=gain_dict.get)
        if divide_fun == Gini_index:
            attr_best = min(gain_dict,key=gain_dict.get)
        anode = Node(root=f'{attr_best}') # maxcnum=len(character_dict[attr_best])
        tree.add(anode)
        # 遍历最优划分属性attr_best的取值
        for attr in character_dict[attr_best]:
            # 为node生成一个分支
            tnode = Node(root={f'{attr_best}={attr}':list(data.loc[data[attr_best]==attr,['编号']].values.flatten())}) # maxcnum=1
            # 令D_v表示D在attr上取值为attr_v的样本子集
            attr_index = [i-1 for i in data.loc[data[attr_best]==attr,['编号']].values.flatten()]
            data_v = data.loc[attr_index,:]
            if len(data_v) == 0: # D_v为空
                # 将分支结点标记为叶结点，其类别标记为D中样本最多的类
                tnode.root[decision_class(data)] = tnode.root.pop(f'{attr_best}={attr}')
                tree.add(tnode)
                continue
#                 return 
            else:
                # 把data_v添加到队列，分支添加到树
                tree.add(tnode)
                character_set = list(set(character_set) - {attr_best})
                queue.append(data_v)
                
def TreeByQueue(data=data0,divide_fun=Gain,MaxDepth=10**4):    
    tree = Tree()
    TreeGenerateByQueue(data=data,tree=tree,divide_fun=divide_fun,MaxDepth=MaxDepth)
    tree.breadth_travel()
    # tree.preorder(tree.root)
    return tree

In [14]:
TreeByQueue(data=data0,divide_fun=Gain,MaxDepth=2)
TreeByQueue(data=data0,divide_fun=Gain,MaxDepth=3)

纹理
{'纹理=稍糊': [7, 9, 13, 14, 17]}
{'纹理=模糊': [11, 12, 16]}
{'纹理=清晰': [1, 2, 3, 4, 5, 6, 8, 10, 15]}
触感
{'触感=硬滑': [9, 13, 14, 17]}
{'触感=软粘': [7]}
{'坏瓜': [11, 12, 16]}
{'坏瓜': [9, 13, 14, 17]}
{'好瓜': [7]}


纹理
{'纹理=稍糊': [7, 9, 13, 14, 17]}
{'纹理=模糊': [11, 12, 16]}
{'纹理=清晰': [1, 2, 3, 4, 5, 6, 8, 10, 15]}
触感
{'触感=硬滑': [9, 13, 14, 17]}
{'触感=软粘': [7]}
{'坏瓜': [11, 12, 16]}
密度
{'密度=大于0.381': [1, 2, 3, 4, 5, 6, 8]}
{'密度=小于0.381': [10, 15]}
{'坏瓜': [9, 13, 14, 17]}
{'好瓜': [7]}
{'好瓜': [1, 2, 3, 4, 5, 6, 8]}
{'坏瓜': [10, 15]}


## 4.8* 试将决策树生成的深度优先搜索过程修改为广度优先搜索，以参数MaxNode控制树的最大结点数，将题 4.7 中基于队列的决策树算法进行改写。对比题 4.7 中的算法，试析哪种方式更易于控制决策树所需存储不超出内存。

沿用上面代码

In [15]:
# 广度优先生成决策树，控制结点数的话，只需要在每次增加结点时记录一次当前结点数，然后判断一次即可
# 这里只控制了分支结点数量
def TreeGenerateBreadthFirst(data,tree=Tree(),divide_fun=Gain,MaxNode=10**4):
    character_set = list(data.columns.drop(['编号','好瓜']))
    # 队列
    queue = [data]
    # 结点
    node_num = 0
    while queue:
        data = queue.pop(0)
        # 生成node 
        dnode = Node(root={'temp':list(data.loc[:,['编号']].values.flatten())}) # maxcnum=1
        # D中样本属于同一类别C
        if len(set(data.loc[:,['好瓜']].values.flatten())) == 1: 
            # 将node标记为C类叶结点
            dnode.root[list(set(data.loc[:,['好瓜']].values.flatten()))[0]] = dnode.root.pop('temp')
            tree.add(dnode)
#             node_num += 1
#             if node_num == MaxNode:
#                 continue
            continue
#             return 
        # character_set为空 or data中样本在attr上取值相同
        if len(character_set) == 0 or len(set([tuple(each) for each in data.loc[:,list(character_set)].values])) <= 1: 
            # 将node标记为叶结点，其类别标记为D中样本数最多的类
            dnode.root[decision_class(data)] = dnode.root.pop('temp')
            tree.add(dnode)
#             node_num += 1
#             if node_num == MaxNode:
#                 continue
            continue
#             return 
        if node_num == MaxNode:
            continue
        # 从A中选择最优划分属性attr_best
        character_dict = {character:attr for character,attr in zip(character_set,[set(data.loc[:,[each]].values.flatten()) for each in character_set])}
        gain_dict = {character:divide_fun(data,character) for character in character_set} 
        attr_best = max(gain_dict,key=gain_dict.get)
        if divide_fun == Gini_index:
            attr_best = min(gain_dict,key=gain_dict.get)
        anode = Node(root=f'{attr_best}') # maxcnum=len(character_dict[attr_best])
        tree.add(anode)
#         node_num += 1
#         if node_num == MaxNode:
#             continue
        # 遍历最优划分属性attr_best的取值
        for attr in character_dict[attr_best]:
            # 为node生成一个分支
            tnode = Node(root={f'{attr_best}={attr}':list(data.loc[data[attr_best]==attr,['编号']].values.flatten())}) # maxcnum=1
            # 令D_v表示D在attr上取值为attr_v的样本子集
            attr_index = [i-1 for i in data.loc[data[attr_best]==attr,['编号']].values.flatten()]
            data_v = data.loc[attr_index,:]
            if len(data_v) == 0: # D_v为空
                # 将分支结点标记为叶结点，其类别标记为D中样本最多的类
                tnode.root[decision_class(data)] = tnode.root.pop(f'{attr_best}={attr}')
#                 node_num += 1
#                 if node_num == MaxNode:
#                     continue
                tree.add(tnode)
                continue
#                 return 
            else:
                # 把data_v添加到队列，分支添加到树
                tree.add(tnode)
                # 判断分支结点数量
                node_num += 1
                if node_num == MaxNode:
                    continue
                character_set = list(set(character_set) - {attr_best})
                queue.append(data_v)
                
def TreeBreadthFirst(data=data0,divide_fun=Gain,MaxNode=10**4):    
    tree = Tree()
    TreeGenerateBreadthFirst(data=data,tree=tree,divide_fun=divide_fun,MaxNode=MaxNode)
    tree.breadth_travel()
    # tree.preorder(tree.root)
    return tree

In [16]:
TreeBreadthFirst(data=data0,divide_fun=Gain,MaxNode=3)
TreeBreadthFirst(data=data0,divide_fun=Gain,MaxNode=5)

纹理
{'纹理=稍糊': [7, 9, 13, 14, 17]}
{'纹理=模糊': [11, 12, 16]}
{'纹理=清晰': [1, 2, 3, 4, 5, 6, 8, 10, 15]}
{'坏瓜': [11, 12, 16]}


纹理
{'纹理=稍糊': [7, 9, 13, 14, 17]}
{'纹理=模糊': [11, 12, 16]}
{'纹理=清晰': [1, 2, 3, 4, 5, 6, 8, 10, 15]}
触感
{'触感=硬滑': [9, 13, 14, 17]}
{'触感=软粘': [7]}
{'坏瓜': [11, 12, 16]}
{'坏瓜': [9, 13, 14, 17]}


## 缺失值处理和多变量决策树日后再补。